In [1]:
import time
import pandas as pd
import numpy as np
import warnings
import feather
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

There are many insignificant features to our work, like codes, PK, FK and out of context. 
Based on prior works and experience of especialists, will be taken off nonsense features. 
A study was carried out on the evolution and correspondence of these resources in the ENEM and Census
data for the period 2009-2018. See (featuresOrganisaton.xls)

In [2]:
teacher = ['FK_COD_DOCENTE', 'PK_COD_ENTIDADE','ID_ESPECIALIZACAO','ID_MESTRADO','ID_DOUTORADO']

school = ['ANO_CENSO','PK_COD_ENTIDADE', 'FK_COD_MUNICIPIO','FK_COD_ESTADO','ID_DEPENDENCIA_ADM','ID_AGUA_INEXISTENTE',
                  'ID_ESGOTO_INEXISTENTE','ID_ENERGIA_INEXISTENTE',
                   'ID_LABORATORIO_INFORMATICA','ID_LABORATORIO_CIENCIAS','ID_SALA_ATENDIMENTO_ESPECIAL',
                   'ID_BIBLIOTECA','ID_SALA_LEITURA','ID_SANITARIO_FORA_PREDIO',
                   'ID_SANITARIO_PNE','ID_DEPENDENCIAS_PNE','NUM_SALAS_UTILIZADAS','ID_EQUIP_TV',
                   'ID_EQUIP_DVD','ID_EQUIP_COPIADORA','ID_EQUIP_IMPRESSORA','NUM_COMP_ADMINISTRATIVOS',
                   'NUM_COMP_ALUNOS','ID_INTERNET','ID_BANDA_LARGA','NUM_FUNCIONARIOS','ID_ALIMENTACAO']

enem = ['NU_ANO','CO_ESCOLA','TP_ENSINO','NU_IDADE','TP_SEXO','Q3','TP_ST_CONCLUSAO','NU_NOTA_CN',
        'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','Q17','Q18','Q15','Q21'
]

         

In [3]:
#Students of ENEM
E = pd.read_csv('~/data/enem/2009/DADOS/MICRODADOS_ENEM_2009.csv', sep=';', usecols = enem )

In [4]:
#School census data
CE =    pd.read_csv("~/data/censo/2009/DADOS/ESCOLAS.CSV", sep='|', encoding="iso-8859-2", usecols = school)

In [5]:
#Teacher census data by Braazilian mesoregion
CD_CO = pd.read_csv("~/data/censo/2009/DADOS/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_NE = pd.read_csv("~/data/censo/2009/DADOS/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_N =  pd.read_csv("~/data/censo/2009/DADOS/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_SE = pd.read_csv("~/data/censo/2009/DADOS/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)
CD_S =  pd.read_csv("~/data/censo/2009/DADOS/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = teacher)

#All teacher census data
CT = pd.concat([CD_CO, CD_NE, CD_N, CD_SE, CD_S])

In [6]:
ENEM = E.copy()
C_SCHOOL = CE.copy()
C_TEACHER = CT.copy()

In [7]:
#ordering
ENEM = ENEM[enem]

newNames = [
'CO_ANO',
'CO_ESCOLA',
'TP_ENSINO',
'NU_IDADE',
'TP_SEXO',
'TP_COR_RACA',
'TP_ST_CONCLUSAO',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT',
'NU_NOTA_REDACAO',
'EDU_PAI',
'EDU_MAE',
'QT_PESSOAS_CASA',
'RENDA_MENSAL'
]

#Rename
ENEM.columns = newNames

print('checking year',ENEM.CO_ANO.unique())
print('checking key',ENEM.CO_ESCOLA.dtypes)

checking year [2009]
checking key float64


In [8]:
C_SCHOOL = C_SCHOOL[school]

newNames = ['ANO_CENSO',
'CO_ESCOLA',
 'CO_MUNICIPIO',
 'CO_UF',
 'CO_DEPENDENCIA_ADM',
 'IN_AGUA_INEXISTENTE',
 'IN_ESGOTO_INEXISTENTE',
 'IN_ENERGIA_INEXISTENTE',
 'IN_LABORATORIO_INFORMATICA',
 'IN_LABORATORIO_CIENCIAS',
 'IN_SALA_ATENDIMENTO_ESPECIAL',
 'IN_BIBLIOTECA',
 'IN_SALA_LEITURA',
 'IN_BANHEIRO_FORA_PREDIO',
 'IN_BANHEIRO_PNE',
 'IN_DEPENDENCIAS_PNE',
 'QT_SALAS_UTILIZADAS',
 'QT_EQUIP_TV',
 'QT_EQUIP_DVD',
 'QT_EQUIP_COPIADORA',
 'QT_EQUIP_IMPRESSORA',
 'QT_COM_ADMINISTRATIVO',
 'QT_COMP_ALUNO',
 'IN_INTERNET',
 'IN_BANDA_LARGA',
 'QT_FUNCIONARIOS',
 'IN_ALIMENTACAO'
]

#Rename 
C_SCHOOL.columns =  newNames

print('Checking year of censo',C_SCHOOL.ANO_CENSO.unique())
C_SCHOOL.drop('ANO_CENSO', axis=1, inplace=True)
print('checking key',C_SCHOOL.CO_ESCOLA.dtypes)

Checking year of censo [2009]
checking key int64


In [9]:
C_TEACHER = C_TEACHER[teacher]

# New default column names for all Datasets
newNames = [
'CO_PROFESSOR',
'CO_ESCOLA',  
'IN_ESPECIALIZACAO',
'IN_MESTRADO',
'IN_DOUTORADO'
]
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
C_TEACHER.columns = newNames
C_TEACHER.drop_duplicates(subset=['CO_PROFESSOR', 'CO_ESCOLA'], inplace=True)
C_TEACHER.drop('CO_PROFESSOR', axis=1, inplace = True)

print('checking key',C_TEACHER.CO_ESCOLA.dtypes)

checking key int64


### Minimum Scope Definition

In [10]:
ENEM.dropna(inplace = True, axis=0, subset = ['CO_ESCOLA'])


In [11]:
#should care about the nulls into scope filter features?
print(ENEM['TP_ENSINO'].value_counts())
print(ENEM['TP_ENSINO'].isnull().sum())
#NO, IT'S LOWER

1.0    1312571
2.0     143109
3.0      69568
4.0      10748
Name: TP_ENSINO, dtype: int64
26


In [12]:
print('drop out',((ENEM.TP_ENSINO!= 1)|(ENEM.TP_ENSINO!= 3)).sum(), 'who not attend regular and prof approachs schools')
ENEM = ENEM.loc[((ENEM.TP_ENSINO== 1)|(ENEM.TP_ENSINO== 3))]
ENEM.drop('TP_ENSINO', inplace = True, axis =1)

drop out 1536022 who not attend regular and prof approachs schools


In [13]:
fields = (['NU_NOTA_CN', 'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT', 'NU_NOTA_REDACAO'])
for i in fields:
    print ('null:', i, ENEM[i].isnull().sum())
    print ('zero:', i, (ENEM[i]== 0).sum())
    
    #, "and", ENEM.I==0)

null: NU_NOTA_CN 372433
zero: NU_NOTA_CN 0
null: NU_NOTA_CH 372433
zero: NU_NOTA_CH 0
null: NU_NOTA_LC 408948
zero: NU_NOTA_LC 0
null: NU_NOTA_MT 408948
zero: NU_NOTA_MT 0
null: NU_NOTA_REDACAO 0
zero: NU_NOTA_REDACAO 449365


In [14]:
ENEM.dropna(inplace = True, axis=0, subset=fields)
ENEM = ENEM.loc[~(ENEM[fields] == 0).any(axis=1)]

In [15]:
#ONLY STUDENTS AT  17 -19 YEAR OLD. WE WANT TO GET ONLY REGULAR STUDENTS THAT ARE IN LAST YEAR OF SECONDARY SCHOOL. 
#THE MICRODADOS HAVE A FIELD TO CONTROL IT. BUT IT IS NOT CONFIDENCE,DUE A TURN OUT TEAACHERS THAT EXTEND ACADEMIC YEAR
print('null', ENEM.NU_IDADE.isnull().sum())
#firts, fill NA with the mean of student school
ENEM.NU_IDADE = ENEM.groupby(ENEM['CO_ESCOLA'])['NU_IDADE'].apply(lambda x: x.fillna(round(x.mean())))
print('taking off',((ENEM.NU_IDADE <17)|(ENEM.NU_IDADE>19)).sum())
ENEM = ENEM.loc[(ENEM.NU_IDADE >16) &  (ENEM.NU_IDADE<20)]
ENEM.NU_IDADE.value_counts()

null 139
taking off 165472


18.0    420500
19.0    242259
17.0     97583
Name: NU_IDADE, dtype: int64

In [16]:
ENEM[fields].describe()

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO
count,760342.000000,760342.000000,760342.000000,760342.000000,760342.000000
mean,516.904035,516.128790,516.887365,513.541366,609.902518
std,98.995528,100.183538,97.538263,102.971051,135.002168
min,263.300000,300.000000,224.300000,338.700000,250.000000
25%,445.400000,437.500000,447.200000,438.700000,525.000000
50%,514.100000,508.600000,517.500000,496.300000,600.000000
75%,584.900000,588.500000,586.700000,569.900000,700.000000
max,903.200000,887.000000,835.600000,985.100000,1000.000000


### Transform features to default categories

In [17]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL']
ENEM[fields].isnull().sum()

TP_SEXO                0
TP_COR_RACA        58599
EDU_PAI            58599
EDU_MAE            58599
QT_PESSOAS_CASA    58599
RENDA_MENSAL       58599
dtype: int64

In [18]:
# Missing values for each case
ENEM.TP_COR_RACA.fillna('F', inplace = True) #missing to not declared


ENEM.EDU_PAI.fillna('I', inplace = True) # Missing to "I do not know" 
ENEM.EDU_MAE.fillna('I', inplace = True) # Missing to "I do not know"

ENEM.QT_PESSOAS_CASA.fillna(ENEM.QT_PESSOAS_CASA.mode()[0], inplace = True) # Have not a specific note, lets fill w mode

ENEM.RENDA_MENSAL.fillna(ENEM.RENDA_MENSAL.mode()[0], inplace = True) #Have not a specific note, lets fill w mode

In [19]:
for i in fields:
    print(i,ENEM[i].value_counts().index)

TP_SEXO Index(['F', 'M'], dtype='object')
TP_COR_RACA Index(['A', 'B', 'F', 'C', 'D', '.', 'E', '*'], dtype='object')
EDU_PAI Index(['E', 'B', 'C', 'I', 'G', 'D', 'H', 'F', 'A', '.', '*'], dtype='object')
EDU_MAE Index(['E', 'B', 'C', 'G', 'I', 'H', 'D', 'F', '.', 'A', '*'], dtype='object')
QT_PESSOAS_CASA Index(['C', 'D', 'B', 'E', 'A', '.', 'F', '*'], dtype='object')
RENDA_MENSAL Index(['C', 'B', 'D', 'A', 'E', '.', 'F', 'H', 'G', '*'], dtype='object')


In [20]:
for i in fields:
    mode = ENEM[i].mode()[0]
    ENEM[i].replace(['.', '*'], mode, inplace = True)

In [21]:
#Order relation, is better for numeric categories.
categorical = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL']
for i in categorical:
    codes, uniques = pd.factorize(ENEM[i], sort=True)
    ENEM[i] = codes
    

In [22]:
ENEM['EDU_PAI_TEMP'] = ENEM['EDU_PAI']
ENEM['EDU_MAE_TEMP'] = ENEM['EDU_MAE']
ENEM['QT_PESSOAS_CASA_TEMP'] = ENEM['QT_PESSOAS_CASA']
ENEM['RENDA_MENSAL_TEMP'] = ENEM['RENDA_MENSAL']
ENEM['TP_COR_RACA_TEMP'] = ENEM['TP_COR_RACA']


In [23]:
# Paring with default categories

ENEM.loc[ENEM['EDU_PAI']==0, 'EDU_PAI_TEMP']=0
ENEM.loc[ENEM['EDU_PAI']==1, 'EDU_PAI_TEMP']=1
ENEM.loc[ENEM['EDU_PAI']==2, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==3, 'EDU_PAI_TEMP']=2
ENEM.loc[ENEM['EDU_PAI']==4, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==5, 'EDU_PAI_TEMP']=3
ENEM.loc[ENEM['EDU_PAI']==6, 'EDU_PAI_TEMP']=4
ENEM.loc[ENEM['EDU_PAI']==7, 'EDU_PAI_TEMP']=5
ENEM.loc[ENEM['EDU_PAI']==8, 'EDU_PAI_TEMP']=0

ENEM.loc[ENEM['EDU_MAE']==0, 'EDU_MAE_TEMP']=0
ENEM.loc[ENEM['EDU_MAE']==1, 'EDU_MAE_TEMP']=1
ENEM.loc[ENEM['EDU_MAE']==2, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==3, 'EDU_MAE_TEMP']=2
ENEM.loc[ENEM['EDU_MAE']==4, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==5, 'EDU_MAE_TEMP']=3
ENEM.loc[ENEM['EDU_MAE']==6, 'EDU_MAE_TEMP']=4
ENEM.loc[ENEM['EDU_MAE']==7, 'EDU_MAE_TEMP']=5
ENEM.loc[ENEM['EDU_MAE']==8, 'EDU_MAE_TEMP']=0


ENEM.loc[ENEM['QT_PESSOAS_CASA']==0, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==1, 'QT_PESSOAS_CASA_TEMP']=1
ENEM.loc[ENEM['QT_PESSOAS_CASA']==2, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==3, 'QT_PESSOAS_CASA_TEMP']=2
ENEM.loc[ENEM['QT_PESSOAS_CASA']==4, 'QT_PESSOAS_CASA_TEMP']=3
ENEM.loc[ENEM['QT_PESSOAS_CASA']==5, 'QT_PESSOAS_CASA_TEMP']=0


ENEM.loc[ENEM['RENDA_MENSAL']==0, 'RENDA_MENSAL_TEMP']=1
ENEM.loc[ENEM['RENDA_MENSAL']==1, 'RENDA_MENSAL_TEMP']=2
ENEM.loc[ENEM['RENDA_MENSAL']==2, 'RENDA_MENSAL_TEMP']=3
ENEM.loc[ENEM['RENDA_MENSAL']==3, 'RENDA_MENSAL_TEMP']=4
ENEM.loc[ENEM['RENDA_MENSAL']==4, 'RENDA_MENSAL_TEMP']=5
ENEM.loc[ENEM['RENDA_MENSAL']==5, 'RENDA_MENSAL_TEMP']=5
ENEM.loc[ENEM['RENDA_MENSAL']==6, 'RENDA_MENSAL_TEMP']=5
ENEM.loc[ENEM['RENDA_MENSAL']==7, 'RENDA_MENSAL_TEMP']=0

ENEM.loc[ENEM['TP_COR_RACA']==0, 'TP_COR_RACA_TEMP']=1
ENEM.loc[ENEM['TP_COR_RACA']==1, 'TP_COR_RACA_TEMP']=3
ENEM.loc[ENEM['TP_COR_RACA']==2, 'TP_COR_RACA_TEMP']=2
ENEM.loc[ENEM['TP_COR_RACA']==3, 'TP_COR_RACA_TEMP']=4
ENEM.loc[ENEM['TP_COR_RACA']==4, 'TP_COR_RACA_TEMP']=5
ENEM.loc[ENEM['TP_COR_RACA']==5, 'TP_COR_RACA_TEMP']=0


In [24]:
#back to original columns
ENEM['EDU_PAI'] = ENEM['EDU_PAI_TEMP']
ENEM['EDU_MAE'] = ENEM['EDU_MAE_TEMP']
ENEM['QT_PESSOAS_CASA'] = ENEM['QT_PESSOAS_CASA_TEMP']
ENEM['RENDA_MENSAL'] = ENEM['RENDA_MENSAL_TEMP']
ENEM['TP_COR_RACA'] = ENEM['TP_COR_RACA_TEMP']

In [25]:
#drop temps

ENEM.drop(['EDU_PAI_TEMP', 'EDU_MAE_TEMP', 'QT_PESSOAS_CASA_TEMP','RENDA_MENSAL_TEMP', 'TP_COR_RACA_TEMP'], axis=1, inplace=True)

In [26]:
print('Total Students at ENEM:', ENEM.shape[0], 'of', ENEM.drop_duplicates('CO_ESCOLA').shape[0], 'schools')
print('Total Schools at Census:', C_SCHOOL.shape[0])

#######################################
print('########### Only about Schools at ENEM:')

schoolsEnem = ENEM['CO_ESCOLA'].dropna().unique()
schoolsCenso = C_SCHOOL['CO_ESCOLA'].dropna().unique()
schoolsTeacher = C_TEACHER['CO_ESCOLA'].dropna().unique()
schoolsEnem2 = ENEM['CO_ESCOLA'].dropna()

enem_censo = np.setdiff1d(schoolsEnem ,schoolsCenso)
print('Schools at ENEM out of School Census', sum(np.isin(schoolsEnem, schoolsCenso, invert=True)))

print('Schools at ENEM out of Teacher Census', sum(np.isin(schoolsEnem, schoolsTeacher, invert=True)))
print('Schools at ENEM and School Census out of Teacher Census', sum(np.isin(enem_censo, schoolsTeacher)))

print('Students Wasted*****', sum(np.isin(schoolsEnem2, schoolsCenso, invert=True)))

     

Total Students at ENEM: 760342 of 25807 schools
Total Schools at Census: 255445
########### Only about Schools at ENEM:
Schools at ENEM out of School Census 0
Schools at ENEM out of Teacher Census 2326
Schools at ENEM and School Census out of Teacher Census 0
Students Wasted***** 0


### New Features

ENEM

In [27]:
#Final Media
ENEM['NU_NOTA_GERAL'] = (ENEM.NU_NOTA_CN + ENEM.NU_NOTA_CH + ENEM.NU_NOTA_LC + ENEM.NU_NOTA_MT + ENEM.NU_NOTA_REDACAO)/5

SCHOOL CENSUS - Itś better include after merge all datasets years.

TEACHER CENSUS

In [28]:
C_TEACHER.isnull().sum()

CO_ESCOLA                 0
IN_ESPECIALIZACAO    795964
IN_MESTRADO          795964
IN_DOUTORADO         795964
dtype: int64

In [29]:
#lets fill it with lower level
C_TEACHER['IN_ESPECIALIZACAO'].fillna(0, inplace = True)
C_TEACHER['IN_MESTRADO'].fillna(0, inplace = True)
C_TEACHER['IN_DOUTORADO'].fillna(0, inplace = True)

In [30]:
C_TEACHER['TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 0

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==0.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 1

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==1.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_ESPECIALIZACAO==0.0) & (C_TEACHER.IN_MESTRADO==1.0)& 
             (C_TEACHER.IN_DOUTORADO ==0.0), 'TITULACAO'] = 2

C_TEACHER.loc[(C_TEACHER.IN_DOUTORADO ==1.0), 'TITULACAO'] = 3

#Rebuild level education information as indicator of unique level.
C_TEACHER['NU_GRADUACAO'] = 0
C_TEACHER['NU_ESPECIALIZACAO'] = 0
C_TEACHER['NU_MESTRADO'] = 0
C_TEACHER['NU_DOUTORADO'] = 0

C_TEACHER.loc[C_TEACHER.TITULACAO == 0, 'NU_GRADUACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 1, 'NU_ESPECIALIZACAO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 2, 'NU_MESTRADO']= 1
C_TEACHER.loc[C_TEACHER.TITULACAO == 3, 'NU_DOUTORADO']= 1


C_TEACHER['NU_GRADUACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_GRADUACAO'].transform(sum)
C_TEACHER['NU_ESPECIALIZACAO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_ESPECIALIZACAO'].transform(sum)
C_TEACHER['NU_MESTRADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_MESTRADO'].transform(sum)
C_TEACHER['NU_DOUTORADO'] = C_TEACHER.groupby('CO_ESCOLA')['NU_DOUTORADO'].transform(sum)



#Calculating weighted indice of teacher education
SUM= C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform(sum)
COUNT =  C_TEACHER.groupby('CO_ESCOLA')['TITULACAO'].transform('count')
C_TEACHER['TITULACAO'] = (SUM)/(COUNT*3)  

##arranjar isso dai



#transformin to school grain
C_TEACHER.drop_duplicates('CO_ESCOLA', inplace = True)

In [31]:
 C_TEACHER.head()

,CO_ESCOLA,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO,NU_GRADUACAO,NU_ESPECIALIZACAO,NU_MESTRADO,NU_DOUTORADO
0,50003577,0.0,0.0,0.0,0.126437,18,11,0,0
27,50007718,0.0,0.0,0.0,0.047619,18,3,0,0
59,50013440,1.0,0.0,0.0,0.222222,8,16,0,0
86,50006410,0.0,0.0,0.0,0.155844,48,22,7,0
119,50006436,1.0,0.0,0.0,0.097561,29,12,0,0


## STUDENT GRAIN

In [32]:
# teacher to Schoool Grain
print('C_SCHOOL',C_SCHOOL.shape)
print('C_TEACHER',C_TEACHER.shape)
CENSO19 = pd.merge(C_SCHOOL, C_TEACHER, on = 'CO_ESCOLA', how = 'inner')
print('C_SCHOOL + C_TEACHER',CENSO19.shape)

#Now, all to Student Grain
print('ENEM',ENEM.shape)
ENEM_CENSO_19 = pd.merge(ENEM, CENSO19, on = 'CO_ESCOLA', how = 'inner')
print('FINAL',ENEM_CENSO_19.shape)

ENEM_CENSO_19.to_csv('STUDENT.csv')

C_SCHOOL (255445, 26)
C_TEACHER (186100, 9)
C_SCHOOL + C_TEACHER (186100, 34)
ENEM (760342, 16)
FINAL (700429, 49)


## SCHOOL GRAIN

In [33]:
fields = ['TP_SEXO', 'TP_COR_RACA', 'EDU_PAI', 'EDU_MAE','QT_PESSOAS_CASA', 'RENDA_MENSAL', 'NU_IDADE', 'NU_NOTA_CN',
             'NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO','NU_NOTA_GERAL' ]

for i in fields:   
    
            ENEM[i] = ENEM.groupby('CO_ESCOLA')[i].transform('mean')
            
            

In [35]:
# SCHOOL GRAIN AND SHOW THE NEW SCORE DISTRIBUTION
ENEM.drop_duplicates('CO_ESCOLA', inplace=True)
#print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')

In [36]:
#recalculating target for new  Nota Geral distribution 
ENEM['TARGET'] = pd.qcut (ENEM.NU_NOTA_GERAL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
print((ENEM.TARGET==0).sum()/(ENEM.TARGET.count())*100, '% lowers quartis')


74.9990312705855 % lowers quartis


In [37]:
print('censo before',CENSO19.shape)
print('enem_school before', ENEM.shape)
CENSO_ENEM_19 = pd.merge(CENSO19, ENEM, on='CO_ESCOLA', how='inner')
print(CENSO_ENEM_19.shape)

CENSO_ENEM_19.to_csv('SCHOOL.csv', index=False)


censo before (186100, 34)
enem_school before (25807, 17)
(23481, 50)


In [38]:
CENSO_ENEM_19.head()

,CO_ESCOLA,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COM_ADMINISTRATIVO,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO,NU_GRADUACAO,NU_ESPECIALIZACAO,NU_MESTRADO,NU_DOUTORADO,CO_ANO,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,NU_NOTA_GERAL,TARGET
0,11007168,1100023,11,2,0.0,0.0,0.0,0,0,0,1,0,1,0,0,13.0,1.0,1.0,0.0,1.0,8.0,NaN,1.0,1.0,95.0,1.0,0.0,0.0,0.0,0.085714,26,9,0,0,2009,18.375000,0.250000,2.125000,2.0,454.556250,438.525000,450.793750,482.962500,504.687500,0.875000,1.562500,2.000000,2.812500,466.305000,0
1,11007885,1100023,11,2,0.0,0.0,0.0,1,1,1,1,0,0,1,1,24.0,1.0,1.0,1.0,1.0,10.0,37.0,1.0,1.0,125.0,1.0,0.0,0.0,0.0,0.066667,45,9,1,0,2009,18.121019,0.401274,2.070064,2.0,497.163694,494.768153,489.412739,482.671338,558.439490,1.821656,2.267516,1.834395,2.547771,504.491083,0
2,11007893,1100023,11,2,0.0,0.0,0.0,1,1,0,1,1,1,1,1,38.0,1.0,1.0,0.0,1.0,6.0,16.0,1.0,1.0,150.0,1.0,1.0,0.0,0.0,0.044444,39,6,0,0,2009,18.068966,0.367816,1.804598,3.0,469.375862,470.108046,466.748276,466.829885,545.689655,1.666667,2.080460,1.896552,2.505747,483.750345,0
3,11034270,1100031,11,2,0.0,0.0,0.0,1,0,0,0,0,0,0,0,5.0,1.0,1.0,0.0,1.0,2.0,10.0,0.0,0.0,13.0,1.0,0.0,0.0,0.0,0.095238,5,2,0,0,2009,18.000000,0.500000,3.000000,2.0,535.650000,514.350000,432.400000,490.900000,537.500000,1.500000,2.000000,1.500000,2.000000,502.160000,0
4,11034440,1100031,11,2,0.0,0.0,0.0,1,0,0,1,1,1,1,0,15.0,1.0,1.0,0.0,1.0,2.0,37.0,1.0,0.0,75.0,1.0,0.0,0.0,0.0,0.131313,20,13,0,0,2009,18.045455,0.454545,2.363636,2.0,502.904545,501.159091,481.750000,476.945455,534.090909,1.590909,2.409091,1.772727,2.454545,499.370000,0


In [39]:
CENSO_ENEM_19.isnull().sum()

CO_ESCOLA                          0
CO_MUNICIPIO                       0
CO_UF                              0
CO_DEPENDENCIA_ADM                 0
IN_AGUA_INEXISTENTE                0
IN_ESGOTO_INEXISTENTE              0
IN_ENERGIA_INEXISTENTE             0
IN_LABORATORIO_INFORMATICA         0
IN_LABORATORIO_CIENCIAS            0
IN_SALA_ATENDIMENTO_ESPECIAL       0
IN_BIBLIOTECA                      0
IN_SALA_LEITURA                    0
IN_BANHEIRO_FORA_PREDIO            0
IN_BANHEIRO_PNE                    0
IN_DEPENDENCIAS_PNE                0
QT_SALAS_UTILIZADAS                0
QT_EQUIP_TV                        0
QT_EQUIP_DVD                       0
QT_EQUIP_COPIADORA                 0
QT_EQUIP_IMPRESSORA                0
QT_COM_ADMINISTRATIVO            720
QT_COMP_ALUNO                   2569
IN_INTERNET                        0
IN_BANDA_LARGA                     0
QT_FUNCIONARIOS                    0
IN_ALIMENTACAO                     0
IN_ESPECIALIZACAO                  0
I

In [40]:
CENSO_ENEM_19.head()

,CO_ESCOLA,CO_MUNICIPIO,CO_UF,CO_DEPENDENCIA_ADM,IN_AGUA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_ENERGIA_INEXISTENTE,IN_LABORATORIO_INFORMATICA,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_PNE,IN_DEPENDENCIAS_PNE,QT_SALAS_UTILIZADAS,QT_EQUIP_TV,QT_EQUIP_DVD,QT_EQUIP_COPIADORA,QT_EQUIP_IMPRESSORA,QT_COM_ADMINISTRATIVO,QT_COMP_ALUNO,IN_INTERNET,IN_BANDA_LARGA,QT_FUNCIONARIOS,IN_ALIMENTACAO,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,TITULACAO,NU_GRADUACAO,NU_ESPECIALIZACAO,NU_MESTRADO,NU_DOUTORADO,CO_ANO,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_ST_CONCLUSAO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,EDU_PAI,EDU_MAE,QT_PESSOAS_CASA,RENDA_MENSAL,NU_NOTA_GERAL,TARGET
0,11007168,1100023,11,2,0.0,0.0,0.0,0,0,0,1,0,1,0,0,13.0,1.0,1.0,0.0,1.0,8.0,NaN,1.0,1.0,95.0,1.0,0.0,0.0,0.0,0.085714,26,9,0,0,2009,18.375000,0.250000,2.125000,2.0,454.556250,438.525000,450.793750,482.962500,504.687500,0.875000,1.562500,2.000000,2.812500,466.305000,0
1,11007885,1100023,11,2,0.0,0.0,0.0,1,1,1,1,0,0,1,1,24.0,1.0,1.0,1.0,1.0,10.0,37.0,1.0,1.0,125.0,1.0,0.0,0.0,0.0,0.066667,45,9,1,0,2009,18.121019,0.401274,2.070064,2.0,497.163694,494.768153,489.412739,482.671338,558.439490,1.821656,2.267516,1.834395,2.547771,504.491083,0
2,11007893,1100023,11,2,0.0,0.0,0.0,1,1,0,1,1,1,1,1,38.0,1.0,1.0,0.0,1.0,6.0,16.0,1.0,1.0,150.0,1.0,1.0,0.0,0.0,0.044444,39,6,0,0,2009,18.068966,0.367816,1.804598,3.0,469.375862,470.108046,466.748276,466.829885,545.689655,1.666667,2.080460,1.896552,2.505747,483.750345,0
3,11034270,1100031,11,2,0.0,0.0,0.0,1,0,0,0,0,0,0,0,5.0,1.0,1.0,0.0,1.0,2.0,10.0,0.0,0.0,13.0,1.0,0.0,0.0,0.0,0.095238,5,2,0,0,2009,18.000000,0.500000,3.000000,2.0,535.650000,514.350000,432.400000,490.900000,537.500000,1.500000,2.000000,1.500000,2.000000,502.160000,0
4,11034440,1100031,11,2,0.0,0.0,0.0,1,0,0,1,1,1,1,0,15.0,1.0,1.0,0.0,1.0,2.0,37.0,1.0,0.0,75.0,1.0,0.0,0.0,0.0,0.131313,20,13,0,0,2009,18.045455,0.454545,2.363636,2.0,502.904545,501.159091,481.750000,476.945455,534.090909,1.590909,2.409091,1.772727,2.454545,499.370000,0
